In [1]:
# Imports
import pandas as pd
import tfx
from tfx.proto import example_gen_pb2
from tfx.components import FileBasedExampleGen, StatisticsGen
from tfx.components.example_gen.custom_executors import parquet_executor
from tfx.dsl.components.base import executor_spec
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.types import standard_component_specs

from tensorflow.core.example.example_pb2 import Example
from tensorflow.python.data import TFRecordDataset
import os

import pprint
pp = pprint.PrettyPrinter()

2022-02-05 15:39:06.993377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-05 15:39:06.993434: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
_trainer_module_file = '/home/mlops/project/test/clean_TFX_trainer.py'

In [3]:
%%writefile {_trainer_module_file}
from tensorflow.python.keras.metrics import BinaryAccuracy
from tensorflow.python.keras.losses import BinaryCrossentropy
from tensorflow.python.keras.optimizer_v2.adam import Adam


from typing import Text, Tuple

import numpy as np
import tensorflow_decision_forests as tfdf
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.components.trainer.fn_args_utils import DataAccessor, FnArgs
from tfx.utils import io_utils
from tfx_bsl.tfxio import dataset_options
import pandas as pd

_FEATURE_KEYS = [
    'spotify_popularity','spotify_followers'
]

_LABEL_KEY = 'spotify_genres_1_index'



_TRAIN_DATA_SIZE = 13600
_TRAIN_BATCH_SIZE = 20


def _input_fn(
    file_pattern: Text,
    data_accessor: DataAccessor,
    schema: schema_pb2.Schema,
    batch_size: int = 20,
    ) -> pd.DataFrame:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: input tfrecord file pattern.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: the number of records to combine in each batch.

  Returns:
    A (features, indices) tuple where features is a matrix of features, and
      indices is a single vector of label indices.
  """
  record_batch_iterator = data_accessor.record_batch_factory(
      file_pattern,
      dataset_options.RecordBatchesOptions(batch_size=batch_size,
                                           num_epochs=1), schema)

  record_df = pd.DataFrame()
  features_list = _FEATURE_KEYS + [_LABEL_KEY]
  for record_batch in record_batch_iterator:
    record_dict = {}
    for column, field in zip(record_batch, record_batch.schema):
      if field.name in features_list:
        record_dict[field.name] = column.flatten()

    record_df = record_df.append( pd.DataFrame(record_dict))

  return record_df


def _train(fn_args, train_df, eval_df):
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label=_LABEL_KEY)
    eval_ds = tfdf.keras.pd_dataframe_to_tf_dataset(eval_df, label=_LABEL_KEY)

    # Train a Random Forest model.
    model = tfdf.keras.RandomForestModel()
    model.fit(train_ds)


    # Summary of the model structure.
    model.summary()
    model.compile(
      metrics=[BinaryAccuracy()])

    model.evaluate(eval_ds)

    return model


# TFX Trainer will call this function.
def run_fn(fn_args: FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  schema = io_utils.parse_pbtxt_file(fn_args.schema_file, schema_pb2.Schema())

  train_df = _input_fn(fn_args.train_files, fn_args.data_accessor,
                               schema)
  eval_df = _input_fn(fn_args.eval_files, fn_args.data_accessor, schema)
  model = _train(fn_args, train_df, eval_df)
  #absl.logging.info(model)


  model.save(fn_args.serving_model_dir, save_format='tf')

Overwriting /home/mlops/project/test/clean_TFX_trainer.py


In [4]:


output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6)
        ,example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))
input_config = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='parquet',
                                pattern='*.parquet'),
])

example_gen = FileBasedExampleGen(
    custom_executor_spec=executor_spec.ExecutorClassSpec(
        parquet_executor.Executor),
    input_base="/home/mlops/project/DeltaLake/platinum_data/ticketmaster_rdy_to_serve",
    input_config=input_config,
    output_config=output_config).with_id('ParquetExampleGen')


context = InteractiveContext(
  pipeline_name='ConcertsPipe',
  pipeline_root='/home/mlops/project/TFX'
  )
context.run(example_gen)


artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /home/mlops/project/TFX/ParquetExampleGen/examples/256


In [5]:

train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "spotify_followers"
    value {
      float_list {
        value: 63242.0
      }
    }
  }
  feature {
    key: "spotify_genres_1_index"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "spotify_popularity"
    value {
      float_list {
        value: 60.0
      }
    }
  }
}

features {
  feature {
    key: "spotify_followers"
    value {
      float_list {
        value: 56211664.0
      }
    }
  }
  feature {
    key: "spotify_genres_1_index"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "spotify_popularity"
    value {
      float_list {
        value: 96.0
      }
    }
  }
}

features {
  feature {
    key: "spotify_followers"
    value {
      float_list {
        value: 88339.0
      }
    }
  }
  feature {
    key: "spotify_genres_1_index"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "spotify_popularity"
    value {
      f

2022-02-05 15:39:19.860209: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-05 15:39:19.860269: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-05 15:39:19.860299: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mlops): /proc/driver/nvidia/version does not exist
2022-02-05 15:39:19.862088: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 257
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 175
        type_id: 16
        uri: "/home/mlops/project/TFX/StatisticsGen/statistics/257"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [31]:
context.show(statistics_gen.outputs['statistics'])

In [32]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 250
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 129
        type_id: 18
        uri: "/home/mlops/project/TFX/SchemaGen/schema/157"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1643759818870
        last_update_time_since_epoch: 1643759818886
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [33]:
context.show(schema_gen.outputs['schema'])

schema_gen.outputs['schema'].get()[0]


,Type,Presence,Valency,Domain
Feature name,,,,
'spotify_followers',FLOAT,required,single,-
'spotify_genres_1_index',INT,required,single,-
'spotify_popularity',FLOAT,required,single,-


.type,<class 'tfx.types.standard_artifacts.Schema'>
.uri,/home/mlops/project/TFX/SchemaGen/schema/157


In [34]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 251
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 130
        type_id: 20
        uri: "/home/mlops/project/TFX/ExampleValidator/anomalies/158"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1643759818959
        last_update_time_since_epoch: 1643759818976
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [35]:
context.show(example_validator.outputs['anomalies'])


In [36]:
from tfx.v1.proto import TrainArgs, EvalArgs
from tensorflow.python.keras.estimator import model_to_estimator

# model = tfdf.keras.RandomForestModel()


trainer = tfx.components.Trainer(
    module_file='/home/mlops/project/test/clean_TFX_trainer.py',
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema']
)
context.run(trainer)

Use /tmp/tmpncqbvfs6 as temporary training directory
Starting reading the dataset
1/1 [==============================] - ETA: 0s
Dataset read in 0:00:04.157440
Training model
Model trained in 0:00:00.168694
Compiling model
1/1 [==============================] - 4s 4s/step
Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (2):
	spotify_followers
	spotify_popularity

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.            "__LABEL"  7.193356 ################
    2. "spotify_popularity"  1.176467 #
    3.  "spotify_followers"  0.400704 

Variable Importance: NUM_AS_ROOT:
    1.  "spotify_followers" 212.000000 ################
    2. "spotify_popularity" 88.000000 

Varia

INFO:tensorflow:Assets written to: /home/mlops/project/TFX/Trainer/model/252/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/mlops/project/TFX/Trainer/model/252/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 252
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 169
        type_id: 22
        uri: "/home/mlops/project/TFX/Trainer/model/252"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 170
        type_id: 23
        uri: "/home/mlops/project/TFX/Trainer/model_run/252"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [37]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

['Format-Serving']
['assets', 'saved_model.pb', 'variables', 'keras_metadata.pb']


In [38]:
model_run_artifact_dir = trainer.outputs['model'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir} --bind_all --port=8008

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

In [39]:
# %reload_ext tensorboard

In [40]:
from google.protobuf import text_format
from tfx.types.standard_artifacts import Model, ModelBlessing
import tfx
import tensorflow_model_analysis as tfma
from tfx.v1.dsl import Resolver, Channel
from tfx.v1.dsl.experimental import LatestBlessedModelStrategy



model_resolver = Resolver(
      strategy_class=LatestBlessedModelStrategy,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing)
).with_id('latest_blessed_model_resolver')

# model_resolver.outputs['model']
context.run(model_resolver)



eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='spotify_genres_1_index')],
      slicing_specs=[
          # An empty slice spec means the overall slice, i.e. the whole dataset.
          tfma.SlicingSpec(),
          # Calculate metrics for each penguin species.
          tfma.SlicingSpec(feature_keys=['spotify_popularity','spotify_followers']),
          ],
      metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.4}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})
                  ))
            ]
        )
    ],
    )

evaluator = tfx.components.Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)

context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 254
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 171
        type_id: 26
        uri: "/home/mlops/project/TFX/Evaluator/evaluation/254"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 172
        type_id: 27
        uri: "/home/mlops/project/TFX/Evaluator/blessing/254"
        custom_properties {
          key: "baseline_model"
          value {
            string_value: "/home/mlops/project/TFX/Trainer/model/207"
          }
        }
        custom_properties {
          key: "baseline_model_id"
          value {
            int_value: 151
          }
        }
        custom_properties {
          key: "blessed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/mlops/project/TFX/Trainer/model/252"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 169
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 27
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [41]:
context.show(evaluator.outputs['evaluation'])


In [42]:
# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

slicing = tfma.view.render_slicing_metrics(
    tfma_result)

context.show(slicing)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [43]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

metric_validations_per_slice {
  slice_key {
    single_slice_keys {
      column: "spotify_followers"
      float_value: 760031.0
    }
    single_slice_keys {
      column: "spotify_popularity"
      float_value: 76.0
    }
  }
  failures {
    metric_key {
      name: "binary_accuracy"
      model_name: "candidate"
      is_diff: true
      example_weighted {
      }
    }
    metric_threshold {
      change_threshold {
        absolute {
          value: -1e-10
        }
        direction: HIGHER_IS_BETTER
      }
    }
    metric_value {
      double_value {
        value: -0.11111111111111116
      }
    }
  }
}
metric_validations_per_slice {
  slice_key {
    single_slice_keys {
      column: "spotify_followers"
      float_value: 243001.0
    }
    single_slice_keys {
      column: "spotify_popularity"
      float_value: 58.0
    }
  }
  failures {
    metric_key {
      name: "binary_accuracy"
      model_name: "candidate"
      is_diff: true
      example_weighted {
      }
 

In [44]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

total 0
-rw-rw-r-- 1 mlops mlops 0 Feb  2 09:39 NOT_BLESSED


In [45]:
_serving_model_dir = '/home/mlops/project/served-model'

In [46]:
from tfx.proto.pusher_pb2 import PushDestination

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=PushDestination(
        filesystem=PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 255
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 173
        type_id: 31
        uri: "/home/mlops/project/TFX/Pusher/pushed_model/255"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [47]:
pusher.outputs


{'pushed_model': Channel(
     type_name: PushedModel
     artifacts: [Artifact(artifact: id: 173
 type_id: 31
 uri: "/home/mlops/project/TFX/Pusher/pushed_model/255"
 custom_properties {
   key: "name"
   value {
     string_value: "pushed_model"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Pusher"
   }
 }
 custom_properties {
   key: "pushed"
   value {
     int_value: 0
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.5.0"
   }
 }
 state: LIVE
 , artifact_type: id: 31
 name: "PushedModel"
 )]
     additional_properties: {}
     additional_custom_properties: {}
 )}

In [48]:
from tensorflow.python.saved_model import saved_model

push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)


OSError: SavedModel file does not exist at: /home/mlops/project/TFX/Pusher/pushed_model/255/{saved_model.pbtxt|saved_model.pb}